In [5]:
import math
from training_functions import *
from merging import *

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading

In [2]:
df_mpdb_plane, cnt_and_name_features_mpdb = get_featurizers_features_mpdb()
df_kahle_fin, cnt_and_name_features_kahle = get_featurizers_features()


In [13]:
X_mpdb = df_mpdb_plane.drop([ 
     'formula_pretty',
     'material_id', 
 ], axis=1)


X = df_kahle_fin.drop([ 
    'stru_label',
     'stru_id', 
     'group', 
     'structure', 
     'diffusion_mean_cm2_s', 
     'label', 
     'src_id', 
     'diffusion_sem_cm2_s', 
     'n_Li', 
     'sigma_S_cm', 
     'sigma_S_cm_sem', 
     'sigma_S_cm_err'
 ], axis=1)


feature_names = X.columns.tolist()



In [15]:
X = X.to_numpy()
X_mpdb = X_mpdb.to_numpy()


feature_names = np.array(feature_names)[~np.all(X == 0, axis=0)]

X_mpdb = X_mpdb[:, ~np.all(X == 0, axis=0)]

X = X[:, ~np.all(X == 0, axis=0)]

feature_weights = [1 / cnt_and_name_features_kahle['_'.join(x.split('_')[1:])] for x in feature_names]

In [20]:
y = (df_kahle_fin['sigma_S_cm'] >= 1e-1).to_numpy()

In [48]:
data = np.concatenate((X, X_mpdb), axis = 0)
labels = np.concatenate((y, -np.ones(X_mpdb.shape[0])), axis = 0)

In [55]:
label_prop_model = LabelSpreading(kernel='knn', tol=0.01, gamma=2)
label_prop_model.fit(data, labels);

In [56]:
predictions = label_prop_model.predict_proba(X_mpdb)

/usr/local/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:230: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [59]:
label_prop_model = LabelPropagation(kernel='rbf', tol=0.01, gamma= 1e-99)
label_prop_model.fit(data, labels);

predictions = label_prop_model.predict_proba(X_mpdb)